Finding Eigenvalues, Eigenvectors plays a crucial role in many places from data science to numerical analysis.                                             
The "pen-paper" way to find eigenvectors and eigenvalues is to find the characteristic matrix via-                                                          $$
\det\!\bigl(A - \lambda I\bigr) \;=\; 0
$$
$$
\det
\begin{pmatrix}
a_{11}-\lambda & a_{12}         & a_{13} \\
a_{21}         & a_{22}-\lambda & a_{23} \\
a_{31}         & a_{32}         & a_{33}-\lambda
\end{pmatrix}
=0
$$
$$
On expanding this we get \lambda^3-(a_{11}+a_{22}+a_{33})\lambda^2+(a_{
$$

Let’s try to find the most **dominant** eigenvalue, by **dominant** we mean the $\lambda_j$ such that $\forall i\neq j,\;|\lambda_i|\le|\lambda_j|$.

In [14]:
import numpy as np

def power_method(A, x0, iterations=6):
    x = x0.astype(float)
    prev_norm = np.linalg.norm(x)
    for k in range(1, iterations+1):
        x = A @ x
        curr_norm = np.linalg.norm(x)
        lam = curr_norm / prev_norm
        v = x / curr_norm       # normalized eigenvector estimate
        print(f"Iteration {k}")
        print(f"  Estimated Eigenvalue λ_{k} ≈ {lam:.4f}")
        print(f"  v_{k} ≈ {v}\n")
        prev_norm = curr_norm
    return x/np.linalg.norm(x)

if __name__ == "__main__":
    A = np.array([[ 2, -12],
                  [ 1,  -5]])
    x0 = np.array([1, 1])
    x_f=power_method(A, x0)
    print(x_f)
    

Iteration 1
  Estimated Eigenvalue λ_1 ≈ 7.6158
  v_1 ≈ [-0.92847669 -0.37139068]

Iteration 2
  Estimated Eigenvalue λ_2 ≈ 2.7606
  v_2 ≈ [0.94174191 0.3363364 ]

Iteration 3
  Estimated Eigenvalue λ_3 ≈ 2.2762
  v_3 ≈ [-0.9456866  -0.32507977]

Iteration 4
  Estimated Eigenvalue λ_4 ≈ 2.1214
  v_4 ≈ [0.94728103 0.32040388]

Iteration 5
  Estimated Eigenvalue λ_5 ≈ 2.0573
  v_5 ≈ [-0.94800398 -0.31825848]

Iteration 6
  Estimated Eigenvalue λ_6 ≈ 2.0278
  v_6 ≈ [0.94834884 0.31722937]

[0.94834884 0.31722937]


In [8]:
print(np.linalg.norm([0.94834884,0.31722937]))

0.9999999977599712


# Power Method — Quick Intuition

What's happening?
An eigenvector $$\vec v$$ satisfies  
$$A\vec v=\lambda\vec v$$  
so applying $A$ repeatedly to any $\vec x_0$ amplifies the dominant direction.

---

Start with $\vec x_0$, then  
$$\vec x_k=A^k\vec x_0.$$  
If $|\lambda_1|>|\lambda_i|$ for all $i\neq1$, then $\vec x_k$ → direction of the dominant eigenvector.

Estimate the dominant eigenvalue by the **growth ratio**:  
$$
\lambda_k\approx\frac{\|\vec x_k\|}{\|\vec x_{k-1}\|}.
$$

Normalize $\vec x_k$ (e.g.\ divide by its norm) each step to keep values stable.

---

**Summary**  
- Multiply: $\vec x_k=A\vec x_{k-1}$.  
- Ratio: $\|\vec x_k\|/\|\vec x_{k-1}\|\to\lambda_1$.  
- Normalize to track the eigenvector direction.  


**Assignment** Can you think of a clever approach to find the second largest eigenvalue, think about **removing** the effect of the most **dominant** eigenvalue/vector pair....

<details>
  <summary>Show power-method code</summary>

  ```python
  import numpy as np

  def power_method(A, x0, iterations=20):
      x = x0 / np.linalg.norm(x0)
      for _ in range(iterations):
          x = A @ x
          x = x / np.linalg.norm(x)
      lam = float(x @ (A @ x))
      return x, lam

  A = np.array([[ 2, -1,  0],
                [-1,  2, -1],
                [ 0, -1,  2]], float)
  v1, λ1 = power_method(A, np.random.rand(3))
  A1    = A - λ1 * np.outer(v1, v1)
  v2, λ2 = power_method(A1, np.random.rand(3))

  print("λ1 =", λ1)
  print("λ2 =", λ2)


Say you think $A$ has an eigenvalue near 3. Do this:

1. Pick the number 3 as your **shift**: set $\mu = 3$.  
2. Build a new matrix $M = (A - 3I)^{-1}$.  
3. Now, in $M$, the eigenvalue that was closest to 3 in $A$ becomes the **biggest** eigenvalue in $M$.  
4. Run the usual Power/Inverse method on $M$—it will quickly find that eigenvalue and its vector.


In [26]:
import numpy as np

def shift_invert_method(A, x0, mu, iterations=20):
    """
    Inverse iteration (shift-invert) to find an eigenpair of A near shift mu.
    Returns approximate eigenvector v and eigenvalue lambda ≈ v^T A v.
    """
    n = A.shape[0]
    B = A - mu * np.eye(n)
    x = x0.astype(float)
    x /= np.linalg.norm(x)

    for _ in range(iterations):
        # Solve (A - mu I) y = x
        y = np.linalg.solve(B, x)
        # Normalize
        x = y / np.linalg.norm(y)

    # Rayleigh quotient gives the eigenvalue of A
    lam = float(x @ (A @ x))
    return x, lam

if __name__ == "__main__":
    A = np.array([[ 2, -12],
                  [ 1,  -5]])
    x0 = np.array([1, 1])
    mu = 0.0   # e.g. target eigenvalue near 0
    v, λ = shift_invert_method(A, x0, mu, iterations=10)
    print("Approximate eigenvector:", v)
    print("Approximate eigenvalue:", λ)


Approximate eigenvector: [-0.97016342 -0.24245194]
Approximate eigenvalue: -0.9988785766190857


In [32]:
print(np.dot(A,v))
print(λ*v)
print(np.linalg.norm((np.dot(A,v))-λ*v))

[0.96909639 0.24209626]
[0.96907545 0.24218004]
8.636064842104585e-05


# Shift-Invert Method — Why It Works

We want an eigenvalue $\lambda_j$ of $A$ **closest** to a chosen shift $\mu$.  Define
$$
M \;=\;(A - \mu I)^{-1}.
$$
If $Av_j=\lambda_j v_j$, then
$$
M v_j \;=\;\frac{1}{\lambda_j - \mu}\,v_j.
$$
Hence the **largest-magnitude** eigenvalue of $M$ corresponds to the $\lambda_j$ nearest to $\mu$.

---

### Algorithm Sketch

1. **Choose** a shift $\mu$ near your target eigenvalue.  
2. **Start** with any nonzero $x_0$.  
3. **Repeat** for $k=1,2,\dots$:
   - Solve 
     $$
     (A - \mu I)\,y = x_{k-1}
     $$
     (e.g.\ via `np.linalg.solve`).
   - Normalize 
     $$
     x_k = \frac{y}{\|y\|}.
     $$
4. **At convergence**, use the Rayleigh quotient to recover  
   $$
   \lambda \approx \frac{x_k^T A\,x_k}{x_k^T x_k}\,.
   $$

---

### Key Points

- **Spectrum flip**: small $|\lambda_j-\mu|$ ⇒ large $1/(\lambda_j-\mu)$ ⇒ dominates in $M$.  
- **No big determinants**: you only solve linear systems and normalize.  
- **Flexible**: by varying $\mu$ you can home in on *any* part of the spectrum.



# Assignment: Applying the Shift-Invert Method

Given the matrix:

$$
A = \begin{bmatrix}
4 & 1 & 0 \\
1 & 3 & 1 \\
0 & 1 & 2 \\
\end{bmatrix}
$$

**Tasks:**

1. Estimate the eigenvalue of $A$ closest to $\mu = 2.5$ using the Shift-Invert Method.
2. Estimate the eigenvalue of $A$ closest to $\mu = 4.5$ using the Shift-Invert Method.

Use an appropriate initial guess for each task and perform several iterations (e.g., 5–10) to approximate the eigenvalue and its corresponding eigenvector.
